In [145]:
#imports and scraping raw data from site
#df is the actual betting line
#df2 has other info, mostly using to get player id to name dict

import pandas as pd
import datetime as dt
import re
from unidecode import unidecode
from betting_functions import get_url_json

In [146]:
dog_props_url = 'https://api.underdogfantasy.com/beta/v3/over_under_lines'
data = get_url_json(dog_props_url)

In [147]:
data.keys()

dict_keys(['appearances', 'games', 'over_under_lines', 'players', 'solo_games'])

In [148]:
df = pd.json_normalize(list(data.values()))
df.index = data.keys()
new = df.T['over_under_lines'].values
lines = pd.json_normalize(new)
lines.sample(2)

,id,live_event,live_event_stat,options,over_under_id,rank,stat_value,status,over_under.id,over_under.appearance_stat.id,over_under.appearance_stat.appearance_id,over_under.appearance_stat.display_stat,over_under.appearance_stat.graded_by,over_under.appearance_stat.pickem_stat_id,over_under.appearance_stat.stat,over_under.boost,over_under.scoring_type_id,over_under.title
2610,52231cc4-1f68-4120-bf50-5a6f94700f96,False,None,[{'id': 'c0fc56b9-c0ae-446d-8ed7-7e659c644e9a'...,7a25551c-aa2f-4bf4-b020-645a2095407a,25180069,14.5,active,7a25551c-aa2f-4bf4-b020-645a2095407a,9a4fa3d7-751f-452a-b724-a7a5c991ec37,9bb500dc-8b03-4bb8-8447-f5b6ecb286d6,Headshots on Maps 1+2,high_score,fe03999c-170d-4ad5-a30c-6ea4eb44bfbd,headshots_on_maps_1_2,None,None,CS: alex (RIDR) Headshots on Maps 1+2 O/U
2648,c56a8b72-8411-4873-950d-861d41d7a938,False,None,[{'id': '15c4ed32-007e-4e35-b68c-fd85ffbaa3fd'...,98a7a858-5a4a-4756-b6b7-680b6d7453fb,25185069,11.5,active,98a7a858-5a4a-4756-b6b7-680b6d7453fb,d25d036d-cf1f-47df-a11c-ebc098dc60b9,01c8d571-0d59-431d-b0dc-615764b5f6ff,Headshots on Maps 1+2,high_score,fe03999c-170d-4ad5-a30c-6ea4eb44bfbd,headshots_on_maps_1_2,None,None,CS: lenci Headshots on Maps 1+2 O/U


In [149]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()#

player_df.loc[player_df.sport_id=='ESPORTS']
x = player_df[player_df.sport_id=='ESPORTS']
p2esport = x[['last_name','first_name']].to_dict('split')
p2esport = dict(p2esport['data'])

In [150]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()
player_df.loc[player_df['sport_id'] != 'ESPORTS', 'full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df.loc[player_df['sport_id'] == 'ESPORTS', 'full_name'] = player_df['last_name']
#player_df['full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df = player_df[['full_name','sport_id','team_id']]
p2l = player_df[['full_name','sport_id']].to_dict('split')
p2tm = player_df[['full_name','team_id']].to_dict('split')
temp = p2l['data']
temp2 = p2tm['data']
p2l = dict(temp)
p2tm = dict(temp2)

In [151]:
player_df

,full_name,sport_id,team_id
0,Giannis Antetokounmpo,NBA,e142b705-dd0a-4803-8cde-a35788a65341
1,Damian Lillard,NBA,e142b705-dd0a-4803-8cde-a35788a65341
2,Brook Lopez,NBA,e142b705-dd0a-4803-8cde-a35788a65341
3,Bobby Portis,NBA,e142b705-dd0a-4803-8cde-a35788a65341
4,Malik Beasley,NBA,e142b705-dd0a-4803-8cde-a35788a65341
...,...,...,...
744,C. Seager + M. Semien,MLBCOMBOS,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8
745,C. Carroll + K. Marte,MLBCOMBOS,db69f844-d6eb-4332-b133-796ce316890a
746,C. Walker + G. Moreno,MLBCOMBOS,db69f844-d6eb-4332-b133-796ce316890a
747,T. Pham + L. Gurriel Jr.,MLBCOMBOS,db69f844-d6eb-4332-b133-796ce316890a


In [152]:
player_df.sport_id.unique()

array(['NBA', 'MLB', 'NHL', 'NFL', 'CFB', 'FIFA', 'SOCCER', 'ESPORTS',
       'NBACOMBOS', 'MLBCOMBOS'], dtype=object)

In [153]:
first2 = player_df['full_name'].apply(lambda x: ' '.join(x.split()[0:2]))
names = pd.DataFrame((first2,player_df.full_name))
names = names.T
names.columns=['first2','fullname']
names.loc[names.first2=='Jazz Chisholm']
first2full = names.to_dict('split')
first2full = dict(first2full['data'])

In [154]:
games = pd.json_normalize(df.T['games'].values).dropna(how='all')
#games = games.loc[games['sport_id']=='NBA']
games.tail(3)

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
71,57255.0,0903e2cd-6363-4592-a410-24fe3a494dde,0.0,a0e08b3e-b870-4204-98b3-0cc5954f9d0a,0.0,Wed 01:00pm,0.0,2023-11-01T17:00:00Z,regular,ESPORTS,scheduled,PKL: Eintracht Spandau vs. Unicorns of Love SE,Game,2023.0
72,57258.0,8bc8be84-70d8-4b7a-91ef-8e41a08b6a79,0.0,501eaaff-16cc-4df5-8f76-f0f82f3b136c,0.0,Wed 01:00pm,0.0,2023-11-01T17:00:00Z,regular,ESPORTS,scheduled,IBCP: KOI Academy vs. UCAM Tokiers,Game,2023.0
73,57259.0,b718ef39-628e-4a03-b075-bce5701748ee,0.0,6474ada3-e859-4092-8649-1f4e527d0b76,0.0,Wed 04:00pm,0.0,2023-11-01T20:00:00Z,regular,ESPORTS,scheduled,IBCP: Giants vs. Rebels Gaming,Game,2023.0


In [155]:
def to_central(date_str):
    x = pd.to_datetime(date_str)
    x = x.tz_convert('US/Central')
    return str(x.date())
to_central('2023-05-20T00:40:00Z')

'2023-05-19'

In [156]:
games.scheduled_at = games.scheduled_at.apply(to_central)

In [157]:
#grabbing team names from match title
matchup = games['title']
home = []
away = []
for i in matchup:
    temp = re.split(' @ |, |vs.',i)
    home.append(temp[0])
    away.append(temp[1])
#fixing esports team names
for i in range(len(home)):
    home[i] = home[i].split(': ')[-1]
    
for i in range(len(away)):
    away[i] = away[i].split(': ')[-1]

matchup = pd.DataFrame((away,home,games.away_team_id,games.home_team_id)).T
matchup.columns = ['Home', 'Away','away_id','home_id']
dates = games['scheduled_at'].values
#temp = []
#for i in dates:
#    temp.append(i.split('T')[0])
matchup['Date'] = dates
id2tm =  matchup[['home_id','Home']].to_dict('split')
id2tmaway =  matchup[['away_id','Away']].to_dict('split')
id2tm = dict(id2tm['data'])
id2tmaway = dict(id2tmaway['data'])
id2tm.update(id2tmaway)

In [158]:
games

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
0,52410.0,e142b705-dd0a-4803-8cde-a35788a65341,0.0,fd41e35e-7b5d-48c3-be0c-69e958976f6b,0.0,Wed 07:00pm,0.0,2023-11-01,regular,NBA,scheduled,MIL @ TOR,Game,2023.0
1,52411.0,0b3fe726-7730-492a-aa50-20413a5c8dae,0.0,47f5f48d-3d14-46f3-b6dc-1e7b4dfb1e69,0.0,Wed 07:00pm,0.0,2023-11-01,regular,NBA,scheduled,POR @ DET,Game,2023.0
2,52414.0,56fcd081-3e21-406e-af95-5716abbfba2e,0.0,c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e,0.0,Wed 07:30pm,0.0,2023-11-01,regular,NBA,scheduled,CLE @ NYK,Game,2023.0
3,52413.0,4da4781a-a2c2-49d7-a042-1dee71740668,0.0,8f5852b6-c8dc-4cce-980a-eab09b28919a,0.0,Wed 07:30pm,0.0,2023-11-01,regular,NBA,scheduled,IND @ BOS,Game,2023.0
4,52412.0,40db32e9-eeb1-4fcf-9e0a-116518312ecf,0.0,f6d5eae0-8f7d-4c63-85b3-b06298f3852c,0.0,Wed 07:30pm,0.0,2023-11-01,regular,NBA,scheduled,NOP @ OKC,Game,2023.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,57320.0,b8881378-98dd-4c9d-98ce-afc6c329a70f,0.0,c693e4bc-f4f9-4618-84c9-b110ac6a2c8e,0.0,Wed 08:00pm,0.0,2023-11-01,regular,ESPORTS,scheduled,CS2: Unjustified vs. Bad New Bears,Game,2023.0
70,57321.0,cca1435b-1ad8-4f61-b6ae-ca871d50a8e1,0.0,b6962a89-01d2-4a5f-992c-dd2e37d20fe4,0.0,Wed 09:00pm,0.0,2023-11-01,regular,ESPORTS,scheduled,CS2: Reason vs. FLUFFY AIMERS,Game,2023.0
71,57255.0,0903e2cd-6363-4592-a410-24fe3a494dde,0.0,a0e08b3e-b870-4204-98b3-0cc5954f9d0a,0.0,Wed 01:00pm,0.0,2023-11-01,regular,ESPORTS,scheduled,PKL: Eintracht Spandau vs. Unicorns of Love SE,Game,2023.0
72,57258.0,8bc8be84-70d8-4b7a-91ef-8e41a08b6a79,0.0,501eaaff-16cc-4df5-8f76-f0f82f3b136c,0.0,Wed 01:00pm,0.0,2023-11-01,regular,ESPORTS,scheduled,IBCP: KOI Academy vs. UCAM Tokiers,Game,2023.0


In [159]:
id2tm

{'fd41e35e-7b5d-48c3-be0c-69e958976f6b': 'TOR',
 '47f5f48d-3d14-46f3-b6dc-1e7b4dfb1e69': 'DET',
 'c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e': 'NYK',
 '8f5852b6-c8dc-4cce-980a-eab09b28919a': 'BOS',
 'f6d5eae0-8f7d-4c63-85b3-b06298f3852c': 'OKC',
 '4ea17ce5-80e9-47b3-9955-2729a02bbe92': 'MIA',
 '04fc51d6-320c-4838-a087-f492067b71f5': 'ATL',
 'f15a9fdd-f178-42ff-aab6-63b9198a34c8': 'MIN',
 '09540cc3-fc27-4a59-b7a6-7a63f0e1df95': 'HOU',
 '2c204562-627d-4fbe-b9ba-24a8bbf87463': 'DAL',
 'ade6b8fc-178f-4e7e-b43d-aa853bea0f3a': 'UTA',
 'd399490e-0df0-4c33-b7c2-fafdadc3a4eb': 'GSW',
 '97616c91-0509-42f8-99f3-04f3856a6377': 'LAL',
 '0331354e-f6cb-4dba-ba55-7c5c40ab9f5e': 'ARI',
 '20a441c7-0cee-48ce-9d26-c09c94a9e594': 'PHI',
 '1d0eee08-3650-4081-bae8-ac7bc3c3cec3': 'CGY',
 '9f5c083d-5d48-4a30-a4d1-f314130878ad': 'COL',
 '9507b4a6-eea0-447d-af29-6391def14b20': 'ANA',
 'ecc8eb1b-f714-57a6-bcf3-b183dd6c12a8': 'PIT',
 'a6f458f4-5078-56e4-a839-af96f1191314': 'KC',
 '01bfe9d5-f671-57ed-aa60-249fcca9267c': 

In [160]:
lines = lines[['stat_value','over_under.appearance_stat.stat', 'over_under.title']]
lines
players = lines['over_under.title'].values
guys = []
for i in players:
    temp = i.split()
    final = temp[0] + ' '+temp[1]
    guys.append(final)
    
guys
lines= lines.rename(columns={'stat_value':'Line','over_under.appearance_stat.stat':'Stat', 'over_under.title':'title'})
lines['Player']= guys
lines['Player'] = lines['Player'].apply(lambda x: unidecode(x).replace('_',' '))


In [161]:
def map_name(x):
    if x is None:
        return None
    if x in first2full:
        return first2full[x]
    elif x.split(': ')[-1] in first2full:
        return x.split(': ')[-1]
    elif x.split()[-1] in first2full:
        return x.split()[-1]
    else:
        return x

lines['Player'] = lines['Player'].apply(lambda x: map_name(x))
lines.Player.unique()

array(['Giannis Antetokounmpo', 'Damian Lillard', 'Brook Lopez',
       'Bobby Portis', 'Malik Beasley', 'Pascal Siakam', 'Scottie Barnes',
       'OG Anunoby', 'Jakob Poeltl', 'Dennis Schroder', 'Scoot Henderson',
       'Jerami Grant', 'Deandre Ayton', 'Shaedon Sharpe',
       'Robert Williams', 'Malcolm Brogdon', 'Matisse Thybulle',
       'Cade Cunningham', 'Killian Hayes', 'Ausar Thompson',
       'Isaiah Stewart', 'Jalen Brunson', 'Julius Randle', 'RJ Barrett',
       'Mitchell Robinson', 'Quentin Grimes', 'Immanuel Quickley',
       'Tyrese Haliburton', 'Myles Turner', 'Bennedict Mathurin',
       'Jayson Tatum', 'Jaylen Brown', 'Kristaps Porzingis',
       'Jrue Holiday', 'Al Horford', 'Derrick White', 'Zion Williamson',
       'Brandon Ingram', 'CJ McCollum', 'Herbert Jones',
       'Jonas Valanciunas', 'Shai Gilgeous-Alexander', 'Chet Holmgren',
       'Josh Giddey', 'Jalen Williams', 'Luguentz Dort', 'Isaiah Joe',
       'Mikal Bridges', 'Ben Simmons', 'Cam Thomas',
       '

In [162]:
league = []
for p in guys:
    if p in p2l:
        league.append(p2l[p])
    elif p.split()[-1] in p2l:
        league.append(p2l[p.split()[-1]])
    else:
        league.append('other')
league[0:5]

['NBA', 'NBA', 'NBA', 'NBA', 'NBA']

In [163]:
guys[1000:1007]

['Ziaire Williams',
 'Xavier Tillman',
 'Ziaire Williams',
 'Ziaire Williams',
 'Xavier Tillman',
 'Ziaire Williams',
 'Xavier Tillman']

In [164]:
team_id = []
for p in guys:
    if p in p2l:
        team_id.append(p2tm[p])
    elif p.split()[-1] in p2l:
        team_id.append(p2tm[p.split()[-1]])
    else:
        team_id.append('other')
team_id[0:5]

['e142b705-dd0a-4803-8cde-a35788a65341',
 'e142b705-dd0a-4803-8cde-a35788a65341',
 'e142b705-dd0a-4803-8cde-a35788a65341',
 'e142b705-dd0a-4803-8cde-a35788a65341',
 'e142b705-dd0a-4803-8cde-a35788a65341']

In [165]:
lines['League'] = league
lines

,Line,Stat,title,Player,League
0,27.5,points,Giannis Antetokounmpo Points O/U,Giannis Antetokounmpo,NBA
1,44.5,pts_rebs_asts,Giannis Antetokounmpo Pts + Rebs + Asts O/U,Giannis Antetokounmpo,NBA
2,5.0,assists,Giannis Antetokounmpo Assists O/U,Giannis Antetokounmpo,NBA
3,11.5,rebounds,Giannis Antetokounmpo Rebounds O/U,Giannis Antetokounmpo,NBA
4,16.5,rebs_asts,Giannis Antetokounmpo Rebounds + Assists O/U,Giannis Antetokounmpo,NBA
...,...,...,...,...,...
2877,15.5,fantasy_points,J. Jung + E. Carter Fantasy Points O/U,J. Jung + E. Carter,other
2878,4.0,hits_runs_rbis,J. Jung + E. Carter Hits + Runs + RBIs O/U,J. Jung + E. Carter,other
2879,3.0,total_bases,J. Jung + E. Carter Total Bases O/U,J. Jung + E. Carter,other
2880,3.0,total_bases,T. Pham + L. Gurriel Jr. Total Bases O/U,T. Pham + L. Gurriel Jr.,other


In [166]:
def map_team(x):
    if x in p2tm:
        return p2tm[x]
    elif p.split()[-1] in p2l:
        return p2tm[p.split()[-1]]
    else:
        return x

lines['team_id'] = lines.Player.apply(lambda x: map_team(x))
lines

,Line,Stat,title,Player,League,team_id
0,27.5,points,Giannis Antetokounmpo Points O/U,Giannis Antetokounmpo,NBA,e142b705-dd0a-4803-8cde-a35788a65341
1,44.5,pts_rebs_asts,Giannis Antetokounmpo Pts + Rebs + Asts O/U,Giannis Antetokounmpo,NBA,e142b705-dd0a-4803-8cde-a35788a65341
2,5.0,assists,Giannis Antetokounmpo Assists O/U,Giannis Antetokounmpo,NBA,e142b705-dd0a-4803-8cde-a35788a65341
3,11.5,rebounds,Giannis Antetokounmpo Rebounds O/U,Giannis Antetokounmpo,NBA,e142b705-dd0a-4803-8cde-a35788a65341
4,16.5,rebs_asts,Giannis Antetokounmpo Rebounds + Assists O/U,Giannis Antetokounmpo,NBA,e142b705-dd0a-4803-8cde-a35788a65341
...,...,...,...,...,...,...
2877,15.5,fantasy_points,J. Jung + E. Carter Fantasy Points O/U,J. Jung + E. Carter,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8
2878,4.0,hits_runs_rbis,J. Jung + E. Carter Hits + Runs + RBIs O/U,J. Jung + E. Carter,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8
2879,3.0,total_bases,J. Jung + E. Carter Total Bases O/U,J. Jung + E. Carter,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8
2880,3.0,total_bases,T. Pham + L. Gurriel Jr. Total Bases O/U,T. Pham + L. Gurriel Jr.,other,db69f844-d6eb-4332-b133-796ce316890a


In [167]:
def map_id(x):
    if x in id2tm:
        return id2tm[x]
    else:
        return x
#lines['team_id'] = lines.team_id.apply(lambda x: map_id(x))


In [168]:
final = pd.merge(lines,matchup, how='left',left_on='team_id',right_on='away_id')
final = final.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2 = pd.merge(final,matchup, how='left',left_on='team_id',right_on='home_id')
final2 = final2.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2['Date'] = final2['Date_x'].fillna(final2['Date_y'])
final2 = final2.drop(['Date_x', 'Date_y'], axis=1)
final2['team_id'] = final2.team_id.apply(lambda x: map_id(x))
final2#.loc[final.Player=='Jimmy Butler']

,Line,Stat,title,Player,League,team_id,Date
0,27.5,points,Giannis Antetokounmpo Points O/U,Giannis Antetokounmpo,NBA,MIL,2023-11-01
1,44.5,pts_rebs_asts,Giannis Antetokounmpo Pts + Rebs + Asts O/U,Giannis Antetokounmpo,NBA,MIL,2023-11-01
2,5.0,assists,Giannis Antetokounmpo Assists O/U,Giannis Antetokounmpo,NBA,MIL,2023-11-01
3,11.5,rebounds,Giannis Antetokounmpo Rebounds O/U,Giannis Antetokounmpo,NBA,MIL,2023-11-01
4,16.5,rebs_asts,Giannis Antetokounmpo Rebounds + Assists O/U,Giannis Antetokounmpo,NBA,MIL,2023-11-01
...,...,...,...,...,...,...,...
2896,15.5,fantasy_points,J. Jung + E. Carter Fantasy Points O/U,J. Jung + E. Carter,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8,NaN
2897,4.0,hits_runs_rbis,J. Jung + E. Carter Hits + Runs + RBIs O/U,J. Jung + E. Carter,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8,NaN
2898,3.0,total_bases,J. Jung + E. Carter Total Bases O/U,J. Jung + E. Carter,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8,NaN
2899,3.0,total_bases,T. Pham + L. Gurriel Jr. Total Bases O/U,T. Pham + L. Gurriel Jr.,other,db69f844-d6eb-4332-b133-796ce316890a,NaN


In [169]:
final2.columns = ['Line', 'Stat', 'title', 'Player', 'League', 'Team', 'Date']

In [170]:
df = final2[['Line', 'Stat', 'Player', 'League', 'Team', 'Date']].copy(deep=True)
df

,Line,Stat,Player,League,Team,Date
0,27.5,points,Giannis Antetokounmpo,NBA,MIL,2023-11-01
1,44.5,pts_rebs_asts,Giannis Antetokounmpo,NBA,MIL,2023-11-01
2,5.0,assists,Giannis Antetokounmpo,NBA,MIL,2023-11-01
3,11.5,rebounds,Giannis Antetokounmpo,NBA,MIL,2023-11-01
4,16.5,rebs_asts,Giannis Antetokounmpo,NBA,MIL,2023-11-01
...,...,...,...,...,...,...
2896,15.5,fantasy_points,J. Jung + E. Carter,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8,NaN
2897,4.0,hits_runs_rbis,J. Jung + E. Carter,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8,NaN
2898,3.0,total_bases,J. Jung + E. Carter,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8,NaN
2899,3.0,total_bases,T. Pham + L. Gurriel Jr.,other,db69f844-d6eb-4332-b133-796ce316890a,NaN


In [171]:
df = df.reset_index(drop=True)

In [172]:
df.Stat.unique()

array(['points', 'pts_rebs_asts', 'assists', 'rebounds', 'rebs_asts',
       'three_points_made', 'pts_asts', 'pts_rebs', 'turnovers',
       'field_goals_att', 'three_points_att', 'personal_fouls',
       'free_throws_made', 'steals', 'blocks', 'blks_stls',
       'double_doubles', 'fantasy_points', 'strikeouts', 'pitch_outs',
       'hits_allowed', 'walks_allowed', 'total_bases', 'runs', 'singles',
       'hits_runs_rbis', 'batter_strikeouts', 'shots', 'blocked_shots',
       'saves', 'goals_against', 'power_play_points', 'goals',
       'passing_yds', 'rushing_yds', 'receiving_rec', 'receiving_yds',
       'rush_rec_yds', 'kicking_points', 'tackles', 'tackles_and_assists',
       'field_goals_made', 'rush_rec_tds', 'passing_tds',
       'extra_points_made', 'passing_and_rushing_yds', 'passing_ints',
       'rushing_att', 'passing_comps', 'passing_att', 'shots_attempted',
       'chances_created', 'shots_on_target', 'fouls', 'aces',
       'double_faults', 'sets_lost', 'breakpoints_w

In [173]:
syntax = {
    'pts_rebs_asts': 'PTS+TRB+AST',
    'pts_rebs': 'PTS+TRB',
    'points': 'PTS',
    'free_throws_made': 'FT',
    'fantasy_points': 'Fantasy',
    'rebounds': 'TRB',
    'assists': 'AST',
    'steals': 'STL',
    'blocks': 'BLK',
    'blks_stls': 'BLK+STL',
    'turnovers': 'TO',
    'rebs_asts': 'TRB+AST',
    'pts_asts': 'PTS+AST',
    'three_points_made': '3PM',
    'shots': 'Shots',
    'goals': 'Goals',
    'saves': 'Goalie Saves',
    'clearances': 'Clearances',
    'goals_and_assists': 'Goals and Assists',
    'tackles': 'Tackles',
    'interceptions': 'Interceptions',
    'crosses': 'Crosses',
    'goals_against': 'Goal Against',
    'game_1_and_2_kills': 'MAPS 1-2 Kills',
    'map_1_and_2_kills': 'MAPS 1-2 Kills',
    'Headshots on Maps 1+2': 'MAPS 1-2 Headshots',
    'strokes': 'Strokes',
    'bogeys_or_worse': 'Bogeys or Worse',
    'birdies_or_better': 'Birdies or Better',
    'aces': 'Aces',
    'breakpoints_won': 'Break Points Won',
    'games_won': 'Games Won',
    'games_lost': 'Games Lost',
    'total_games': 'Total Games',
    'double_faults': 'Double Faults',
    'service_games_lost': 'Service Games Lost',
    'Fastest Laps': 'Fastest Laps',
    'position': 'position',
    'passing_yds': 'Pass Yards',
    'rushing_yds': 'Rush Yards',
    'receiving_yds': 'Receiving Yards' ,
    'fantasy_points': 'Hitter Fantasy Score',
    'strikeouts': 'Pitcher Strikeouts',
    'hits_runs_rbis': 'PTS',
    'singles':'Singles',
    'total_bases': 'Total Bases',
    'walks': 'Walks Allowed',
    'hits': 'Hits Allowed',
                }

In [174]:
df = df.replace(syntax)

In [175]:
df.loc[df['League'] == 'ESPORTS']

,Line,Stat,Player,League,Team,Date
1752,27.5,kills_in_maps_1_2_3,Xiaohu+Light,ESPORTS,NRG,2023-11-02
1753,13.5,kills_in_maps_1_2_3,Theshy+Weiwei,ESPORTS,NRG,2023-11-02
1754,23.8,fantasy_points_on_map_1,TheShy,ESPORTS,NRG,2023-11-02
1755,61.3,fantasy_points_on_maps_1_2_3,TheShy,ESPORTS,NRG,2023-11-02
1756,2.5,kills_in_map_1,TheShy,ESPORTS,NRG,2023-11-02
...,...,...,...,...,...,...
2802,1.0,kills_in_maps_1_2,Kibah,ESPORTS,Rebels Gaming,2023-11-01
2803,4.5,kills_in_maps_1_2,R4VEN,ESPORTS,Rebels Gaming,2023-11-01
2804,5.0,kills_in_maps_1_2,AHaHaCiK,ESPORTS,Rebels Gaming,2023-11-01
2805,7.5,kills_in_maps_1_2,Random,ESPORTS,Rebels Gaming,2023-11-01


In [176]:
df.loc[df.Player=='Krimbo']

,Line,Stat,Player,League,Team,Date


In [177]:
today = dt.date.today()

In [178]:
str(today)

'2023-11-01'

In [179]:
df.columns = [x.lower() for x in df.columns]
df.to_csv(f'Lines/dog/doglines_{today.month}_{today.day}.csv')

In [180]:
df.loc[df.league=='ESPORTS']

,line,stat,player,league,team,date
1752,27.5,kills_in_maps_1_2_3,Xiaohu+Light,ESPORTS,NRG,2023-11-02
1753,13.5,kills_in_maps_1_2_3,Theshy+Weiwei,ESPORTS,NRG,2023-11-02
1754,23.8,fantasy_points_on_map_1,TheShy,ESPORTS,NRG,2023-11-02
1755,61.3,fantasy_points_on_maps_1_2_3,TheShy,ESPORTS,NRG,2023-11-02
1756,2.5,kills_in_map_1,TheShy,ESPORTS,NRG,2023-11-02
...,...,...,...,...,...,...
2802,1.0,kills_in_maps_1_2,Kibah,ESPORTS,Rebels Gaming,2023-11-01
2803,4.5,kills_in_maps_1_2,R4VEN,ESPORTS,Rebels Gaming,2023-11-01
2804,5.0,kills_in_maps_1_2,AHaHaCiK,ESPORTS,Rebels Gaming,2023-11-01
2805,7.5,kills_in_maps_1_2,Random,ESPORTS,Rebels Gaming,2023-11-01
